## Primer Metodo Keras

In [1]:
from sklearn.datasets import dump_svmlight_file
import numpy as np
import pandas as pd
import os
import urllib
import zipfile
import opendatasets as od
from sklearn.model_selection import train_test_split
import shutil
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
dataset = pd.read_csv('clicks_train.csv')
dataset

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0
...,...,...,...
87141726,16874592,186600,0
87141727,16874593,151498,1
87141728,16874593,282350,0
87141729,16874593,521828,0


In [3]:
dataset.display_id = dataset.display_id.astype('category').cat.codes.values
dataset.ad_id = dataset.ad_id.astype('category').cat.codes.values

In [4]:
train, test = train_test_split(dataset, test_size=0.2)

In [5]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

### Error con la reduccion de dimensiones

In [6]:
#n_users, n_movies = len(dataset.display_id.unique()), len(dataset.ad_id.unique())
n_users, n_movies = 10000 , 8000
n_latent_factors = 20

In [7]:
n_users 

10000

In [8]:
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))
prod = keras.layers.dot([movie_vec, user_vec], axes=1,name='DotProduct')
print(prod)
model = keras.Model([user_input, movie_input], prod)

KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='DotProduct/Squeeze:0', description="created by layer 'DotProduct'")


In [9]:
movie_embedding

<KerasTensor: shape=(None, 1, 20) dtype=float32 (created by layer 'Movie-Embedding')>

In [10]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Item (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 User (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 Movie-Embedding (Embedding)    (None, 1, 20)        160020      ['Item[0][0]']                   
                                                                                                  
 User-Embedding (Embedding)     (None, 1, 20)        200020      ['User[0][0]']                   
                                                                                              

In [12]:
tf.keras.utils.plot_model(model, to_file='model.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### Error con la reduccion de dimensiones

In [13]:
train.display_id = train.display_id.astype('category').cat.codes.values
train.ad_id = train.ad_id.astype('category').cat.codes.values

In [14]:
pd.DataFrame(train.display_id)

,display_id
6880443,1321463
35953907,6957688
45414197,8788654
1910760,372489
81002472,15625275
...,...
54373969,10512512
41192513,7983848
83204474,16056091
74150435,14310501


In [27]:
history = model.fit([train.display_id, train.ad_id], train.clicked, epochs=100, verbose=0)

InvalidArgumentError: Graph execution error:

Detected at node 'model/User-Embedding/embedding_lookup' defined at (most recent call last):
    File "C:\Users\pablo\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\pablo\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\pablo\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\pablo\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\pablo\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\pablo\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\pablo\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self

In [28]:
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Training Error")

NameError: name 'history' is not defined

In [ ]:
test.display_id = test.display_id.astype('category').cat.codes.values
test.ad_id = test.ad_id.astype('category').cat.codes.values

In [ ]:
results = model.evaluate((test.display_id, test.ad_id), test.clicked, batch_size=1)

In [ ]:
movie_embedding_learnt = model.get_layer(name='Movie-Embedding').get_weights()[0]
pd.DataFrame(movie_embedding_learnt).describe()

In [ ]:
user_embedding_learnt = model.get_layer(name='User-Embedding').get_weights()[0]

In [ ]:
def recommend(user_id, number_of_movies=5):
    movies = user_embedding_learnt[user_id]@movie_embedding_learnt.T
    mids = np.argpartition(movies, -number_of_movies)[-number_of_movies:]
    return mids

In [ ]:
recommend(user_id=1)

## Segundo metodo Keras

In [16]:
### Ahora en Keras

from __future__ import print_function, division
from builtins import range, input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import tensorflow

from tensorflow import keras

In [17]:
from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

In [18]:
df = pd.read_csv('clicks_train.csv')

In [19]:
df.head()

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0


In [20]:
user = df['display_id'].value_counts().index
map = {k:i for i, k in enumerate(user)}
df['display_id'] = df['display_id'].map(map)

In [21]:
mov = df['ad_id'].value_counts().index
map = {k:i for i, k in enumerate(mov)}
df['ad_id'] = df['ad_id'].map(map)

In [22]:
df

,display_id,ad_id,clicked
0,3708577,1426,0
1,3708577,10810,0
2,3708577,82,1
3,3708577,80,0
4,3708577,22809,0
...,...,...,...
87141726,9786521,944,0
87141727,11566925,22257,1
87141728,11566925,2311,0
87141729,11566925,5476,0


In [23]:
N = df['display_id'].max()
M = df['ad_id'].max()

N,M

(16874592, 478949)

In [24]:
n = 10000
m = 8000

from collections import Counter

In [25]:
ucount = Counter(df['display_id'])
mcount = Counter(df['ad_id'])

uid = [u for u, c in ucount.most_common(n)]
mid = [u for u, c in mcount.most_common(m)]

In [26]:
newdf = df[df['display_id'].isin(uid) & df['ad_id'].isin(mid)]
newdf.head()

MemoryError: Unable to allocate 1.95 GiB for an array with shape (3, 87141731) and data type int64

In [ ]:
N = newdf['display_id'].max()
M = newdf['ad_id'].max()

user = newdf['display_id'].value_counts().index
map = {k:i for i, k in enumerate(user)}
newdf['display_id'] = newdf['display_id'].map(map)

In [ ]:
mov = newdf['ad_id'].value_counts().index
map = {k:i for i, k in enumerate(mov)}
newdf['ad_id'] = newdf['ad_id'].map(map)

In [ ]:
newdf = shuffle(newdf)
cutoff = int(0.8*len(newdf))
cutoff

In [ ]:
train = newdf.iloc[: cutoff]
test = newdf.iloc[cutoff:]

In [ ]:
K = 10 #factores latentes
mu = newdf['clicked'].mean()
epochs = 25
reg = 0.
N = 10000
M = 8000

In [ ]:
u = Input(shape=(1,))
m = Input(shape=(1,))

u_embed = Embedding(N, K, embeddings_regularizer = l2(reg))(u) # size (N,1,K)
m_embed = Embedding(M, K, embeddings_regularizer = l2(reg))(m) # size (M,1,K)

In [ ]:
u_bias = Embedding(N, 1, embeddings_regularizer = l2(reg))(u)
m_bias = Embedding(M, 1, embeddings_regularizer = l2(reg))(m)

x = Dot(axes = 2)([u_embed, m_embed])

x = Add()([x, u_bias, m_bias])
x = Flatten()(x) # N,1

In [ ]:
model = Model(inputs = (u, m),
             outputs = x)

model.compile(loss = 'mse',
             optimizer = Adam(learning_rate = 0.0001),
             metrics = ['mse'])

In [ ]:
r = model.fit(x = [train['display_id'].values, train['ad_id'].values],
                 y = train['clicked'].values - mu,
                 epochs = epochs,
                 batch_size = 256,
                 validation_data = ([test['display_id'].values, test['ad_id'].values],
                 test['clicked'].values - mu)
                 )

In [ ]:
# plot losses
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="test loss")
plt.legend()
plt.show()

# plot mse
plt.plot(r.history['mse'], label="train mse")
plt.plot(r.history['val_mse'], label="test mse")
plt.legend()
plt.show()

## Tercer Metodo Keras

In [ ]:
from sklearn.datasets import dump_svmlight_file
import numpy as np
import pandas as pd
import os
import urllib
import zipfile
from sklearn.model_selection import train_test_split
import shutil
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
### Ahora en Keras

from __future__ import print_function, division
from builtins import range, input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import tensorflow

from tensorflow import keras

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
df = pd.read_csv('clicks_train.csv')

In [ ]:
df.head()

In [ ]:
df.display_id = df.display_id.astype('category').cat.codes.values
df.ad_id = df.ad_id.astype('category').cat.codes.values

In [ ]:
n = 10000
m = 8000

from collections import Counter

In [ ]:
ucount = Counter(df['display_id'])
mcount = Counter(df['ad_id'])

uid = [u for u, c in ucount.most_common(n)]
mid = [u for u, c in mcount.most_common(m)]

In [ ]:
newdf = df[df['display_id'].isin(uid) & df['ad_id'].isin(mid)]
newdf.head()

In [ ]:
newdf.display_id = newdf.display_id.astype('category').cat.codes.values
newdf.ad_id = newdf.ad_id.astype('category').cat.codes.values

In [ ]:
train, test = train_test_split(newdf, test_size=0.2)

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [ ]:
#n_users, n_movies = len(dataset.display_id.unique()), len(dataset.ad_id.unique())
n_users, n_movies = 10000 , 8000
n_latent_factors = 20

In [ ]:
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))
prod = keras.layers.dot([movie_vec, user_vec], axes=1,name='DotProduct')
print(prod)
model = keras.Model([user_input, movie_input], prod)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [ ]:
model.summary()

In [ ]:
history = model.fit([train.display_id, train.ad_id], train.clicked, epochs=25, verbose=0, 
                    validation_data = ([test.display_id, test.ad_id], test.clicked))

In [ ]:
pd.Series(history.history['loss']).plot(logy=True, color='b')
plt.xlabel("Epoch")
plt.ylabel("Training Error")

In [ ]:
pd.Series(history.history['val_loss']).plot(logy=True, color='orange')
plt.xlabel("Epoch")
plt.ylabel("Training Error")

In [ ]:
results = model.evaluate((test.display_id, test.ad_id), test.clicked, batch_size=1)

In [ ]:
movie_embedding_learnt = model.get_layer(name='Movie-Embedding').get_weights()[0]
pd.DataFrame(movie_embedding_learnt).describe()

In [ ]:
user_embedding_learnt = model.get_layer(name='User-Embedding').get_weights()[0]

In [ ]:
def recommend(user_id, number_of_movies=5):
    movies = user_embedding_learnt[user_id]@movie_embedding_learnt.T
    mids = np.argpartition(movies, -number_of_movies)[-number_of_movies:]
    return mids

In [ ]:
recommend(user_id=1)